In [1]:
import os, ast
from graphviz import Digraph

project_root = "src"
graph = Digraph(comment="Python File Dependencies")
graph.attr(rankdir="LR")  # horizontal layout, often cleaner

# Map file path → module name
files = {
    os.path.relpath(os.path.join(dp, f), project_root).replace(os.sep, ".").rstrip(".py"): os.path.join(dp, f)
    for dp, _, filenames in os.walk(project_root)
    for f in filenames if f.endswith(".py")
}

def get_imports(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        tree = ast.parse(f.read(), filename=file_path)
    imports = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Import):
            imports.extend(n.name for n in node.names)
        elif isinstance(node, ast.ImportFrom) and node.module:
            imports.append(node.module)
    return imports

# Group modules by package
packages = {}
for module in files:
    pkg = ".".join(module.split(".")[:-1])  # folder path
    packages.setdefault(pkg, []).append(module)

# Add nodes in clusters
for pkg, mods in packages.items():
    if pkg:  # only cluster if not top-level
        with graph.subgraph(name=f"cluster_{pkg}") as c:
            c.attr(label=pkg)
            for m in mods:
                c.node(m)
    else:
        for m in mods:
            graph.node(m)

# Add edges for internal imports only
for module, path in files.items():
    imports = get_imports(path)
    for imp in imports:
        for target in files:
            if imp == target or imp.startswith(target + "."):
                graph.edge(module, target)

graph.render("file_dependencies", format="svg", view=True)

'file_dependencies.svg'